In [ ]:
import syft as sy

# Part 1: Join the Duet Server the Data Owner connected to

In [ ]:
duet = sy.join_duet(loopback=True)

Duet server provided by Open minded
Checkpoint 0 : Now STOP and run the Data Owner notebook until Checkpoint 1.

# Part 2: Search for Available Data


In [ ]:
# The data scientist can check the list of pointable data in Data Owner's duet store
duet.store.pandas

In [ ]:
# Data Scientist wants to get the Crop dataset. (S)He needs a pointer to the data and
# a pointer to the target for prediction.
data_ptr = duet.store[0]
target_ptr = duet.store[1]

# data_ptr.requires_grad = True
# target_ptr.requires_grad = True

# data_ptr is a reference to the Crop dataset remotely available on data owner's server
# target_ptr is a reference to the Crop dataset LABELS
# remotely available on data owner's server
print(data_ptr)
print(target_ptr)

### Basic analysis

First the data scientist needs to know some basic information about the dataset.
1. The length of the dataset
2. The input dimension
3. The output dimension

These information have to explicitly shared by the Data Owner. Let's try to find them in the data description.

In [ ]:
print(duet.store.pandas["Description"][0])

print(duet.store.pandas["Description"][1])

### Train model

In [ ]:
import torch
import xgboost as xgb
import time
from sklearn.metrics import mean_squared_error

In [ ]:
in_dim = 4
out_dim = 3
n_samples = 150

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.layer1 = self.torch_ref.nn.Linear(in_dim, 20)
        self.layer2 = self.torch_ref.nn.Linear(20, 30)
        self.out = self.torch_ref.nn.Linear(30, out_dim)
        #self.flatten = self.torch_ref.nn.Flatten()
    def forward(self, X111):
        X111 = self.torch_ref.nn.functional.relu(self.layer1(X111))
        X111 = self.torch_ref.nn.functional.relu(self.layer2(X111))
        output = self.torch_ref.nn.functional.log_softmax(self.out(X111), dim=1)
        #self.torch_ref.nn.Flatten(x)
        return output


local_model = SyNet(torch)

In [ ]:
remote_model = local_model.send(duet)

In [ ]:
remote_torch = duet.torch

In [ ]:
remote_torch

In [ ]:
params = remote_model.parameters()
optim = remote_torch.optim.Adam(params=params, lr=0.01)

In [ ]:
print(remote_model.parameters())

In [ ]:
print(optim)

In [ ]:
start_time = time.time()
def train(iterations, model, torch_ref, optim, data_ptr, target_ptr):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data_ptr)

        loss = torch_ref.nn.functional.nll_loss(output, target_ptr.long())

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress", request_block=True, timeout_secs=5
        )

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_value)

        losses.append(loss_value)

        loss.backward()

        optim.step()

    return losses
print("Computational time :- %s seconds " % (time.time() - start_time))

In [ ]:
iteration = 300
losses = train(iteration, remote_model, remote_torch, optim, data_ptr, target_ptr)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.title('Crop Type, Loss vs iteration')
plt.plot(range(iteration), losses)
plt.ylabel("Loss")
plt.xlabel("iteration")


### Download model

In [ ]:
def get_local_model(model):
    if not model.is_local:
        local_model = model.get(
            request_block=True,
            reason="To run test and inference locally",
            timeout_secs=5,
        )
    else:
        local_model = model

    return local_model


local_model = get_local_model(remote_model)

In [ ]:
local_model

### Test on local data

In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
crop_test = pd.read_csv(f"Crop_test_data_Artharva1.csv")

In [ ]:
from sklearn.preprocessing import StandardScaler
df2 = StandardScaler().fit_transform(crop_test.iloc[:, 0:5])

In [ ]:
df = pd.DataFrame(df2)

In [ ]:
XX_test = df.loc[:, crop_test.columns != "label"]
yy_test = crop_test["label"]

In [ ]:
XX_test

In [ ]:
yy_test

In [ ]:
XX1_test = torch.FloatTensor(np.array(XX_test))
yy1_test = torch.LongTensor(np.array(yy_test))

In [ ]:
preds = []
with torch.no_grad():
    #local_model.eval()
    for i in range(len(XX1_test)):
        sample = XX1_test[i]
        #try replacing X_test[i] with X_test[i:i+1]
        y_hat = local_model(sample.unsqueeze(0))
        pred = y_hat.argmax().item()
        #pred = y_hat.max().item()
        #CATEGORIES[np.argmax(pred)]
        print(f"Prediction: {pred} Ground Truth: {yy1_test[i]}")
        preds.append(pred)

In [ ]:
print("Predicted crop is",pred)

In [ ]:
sample

In [ ]:
y_hat

In [ ]:
acc = accuracy_score(yy1_test, preds)
print("Overall test accuracy", acc * 100)

In [ ]:
plt.title('Crop Type Predictions and Ground Truth')

plt.plot(preds, label='pred')
plt.plot(yy1_test, label='actual')
plt.legend()
plt.show()

In [ ]:
#print("Base Model parameters:")
print(remote_model)
print()

#print("Remote model1 parameters:")
#print(remote_torch)
print()

#print("Remote model2 parameters:")
#print(param2)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(yy1_test, preds))